In [1]:
import pandas as pd
%matplotlib qt
import sys
import os
import warnings
from tqdm import tqdm
import numpy as np
warnings.filterwarnings("ignore", category=RuntimeWarning)
sys.path.append('../../')
import Utils.Preprocess as ut
project_path = os.path.abspath(os.path.relpath('../../../../', os.getcwd()))
chb_root_path=os.path.join(project_path,'Dataset/CHB-MIT')
constrain_path=os.path.join(project_path,'BilinearNetwork/Data/Constraint/Prediction')
save_path=os.path.join(project_path,'BilinearNetwork/Data/PreprocessedData/CHB-MIT/Prediction')

In [2]:
def extract_epochs(raw_path,start,stop):
    raw_T=ut.PreprocessTool(raw_path,start=start,stop=stop,duration=5)
    raw_T.do_preprocess().no_overlap_events().cut_epochs()
    epochs_data=raw_T.get_epochs().get_data().astype(np.float32)
    stft_data=raw_T.get_epochs_stft().astype(np.float32)
    del raw_T
    return epochs_data,stft_data
exclude_file=pd.read_csv(os.path.join(constrain_path,'exclude_File.csv'))
exclude_patient=pd.read_csv(os.path.join(constrain_path,'exclude_Patient.csv'))

def find_all_files(chb_root_path):
    all_files=[]
    for root,dirs,files in os.walk(chb_root_path):
        for file in files:
            if file.endswith('.edf'):
                all_files.append(file)
    return all_files

In [3]:
small_constrant=list(exclude_file['File Name'])
exclude_patient=list(exclude_patient['0'])
large_table=find_all_files(chb_root_path)

In [4]:
# 将large_table中small_constrant删除
select_files=large_table.copy()
for file_name in large_table:
    prelix=file_name.split('_')[0]
    if prelix in exclude_patient:
        select_files.remove(file_name)
        continue
    if file_name in small_constrant:
        select_files.remove(file_name)
        continue

In [5]:
# preictal之前的两个文件，作为interictal
select_file_final=[]
for i in range(len(select_files)-1):
    current_num=int(select_files[i].split('_')[1].split('.')[0])
    prelix=select_files[i].split('_')[0]
    next_num=int(select_files[i+1].split('_')[1].split('.')[0])
    if abs(next_num-current_num)>1:
        select_file_final.append(select_files[i])
        previous_file_name=prelix+'_'+str(current_num-1).zfill(2)+'.edf'
        previous_previous_file_name=prelix+'_'+str(current_num-2).zfill(2)+'.edf'
        if previous_file_name in select_files:
            select_file_final.append(previous_file_name)
        if previous_previous_file_name in select_files:
            select_file_final.append(previous_previous_file_name)
#  enhance interictal size mannually
select_file_final.extend(['chb19_07.edf','chb19_10.edf','chb19_15.edf','chb99_06.edf','chb99_01.edf','chb99_02.edf'])
select_file_final.sort()

In [6]:
select_file_final

In [8]:
prelix=None
local_epochs_stft=[]
for file_name in tqdm(select_file_final):
    prelix_old=prelix
    prelix=file_name.split('_')[0]
    if prelix_old!=prelix and prelix!='chb01':
        save_path_stft='%s_interictal.npy'%os.path.join(save_path,prelix_old)
        local_epochs_stft=np.concatenate(local_epochs_stft,axis=0)
        np.save(save_path_stft,local_epochs_stft)
        print(local_epochs_stft.shape)
        print('-------')
        local_epochs_stft=[]

    chb_patient_path=os.path.join(chb_root_path,prelix.rstrip('a').rstrip('b').rstrip('c'))
    process_file_path=os.path.join(chb_patient_path,file_name)
    
    start=10
    # can set huge number, once larger than EEG recording the mne will take the whole period
    stop=6000
    
    try:
        _,local_epoch_stft=extract_epochs(process_file_path,start,stop)
        local_epochs_stft.append(local_epoch_stft)
    except Exception as e:
        print("ERROR")
        print(e)
        break